In [2]:
import os
import cv2
import numpy as np


In [3]:

# define the path to the input images
input_path = r"C:\Users\user\Documents\GitHub\Hand Gesture\Hand-Gesture\input"

# define the path to the output preprocessed images
output_path = r"C:\Users\user\Documents\GitHub\Hand Gesture\Hand-Gesture\output"

# define the target size of the preprocessed images
target_size = (4608, 2592)

# define the preprocessing functions


def convert_to_gray(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.convertScaleAbs(gray)
    return gray


def apply_gaussian_blur(img):
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    return blurred


def apply_threshold(img):
    _, thresh = cv2.threshold(
        img, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return thresh


def find_contours(img):
    contours, hierarchy = cv2.findContours(
        img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours, hierarchy


def find_largest_contour(contours):
    max_contour = max(contours, key=cv2.contourArea)
    return max_contour


def create_mask(img, contour):
    mask = cv2.drawContours(np.zeros_like(
        img), [contour], 0, (255, 255, 255), -1)
    return mask


def apply_mask(img, mask):
    masked_image = cv2.bitwise_and(img, img, mask=mask)
    return masked_image


def resize_image(img, size):
    resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return resized


def normalize_image(img):
    normalized = img.astype("float32") / 255.0
    return normalized


def get_hog_features(img):
    win_size = (64, 64)
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size,
                            block_stride, cell_size, nbins)
    hog_feats = hog.compute(img)
    return hog_feats


def get_sift_features(img):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return descriptors



In [ ]:

all_feats = []
all_imgs = []


# load the image
img = cv2.imread("./test.jpg")
print(img)

# apply the preprocessing pipeline
gray = convert_to_gray(img)
blurred = apply_gaussian_blur(gray)
thresh = apply_threshold(blurred)
contours, hierarchy = find_contours(thresh)
max_contour = find_largest_contour(contours)
mask = create_mask(gray, max_contour)
masked_image = apply_mask(img, mask)
resized = resize_image(masked_image, target_size)
normalized = normalize_image(resized)
print(normalized)
all_imgs.append(normalized)
# loop through all_imgs and extract features for each image
for img in all_imgs:

    imgH = convert_to_gray(img)
    
    print(imgH)
    if imgH.ndim == 2:
        hog_feats = get_hog_features(imgH)
    else:
        hog_feats = get_hog_features(imgH[:, :, 0])

    # extract SIFT features
    sift_feats = get_sift_features(imgH)
    if sift_feats is None:
         sift_feats = np.array([])

    # concatenate HoG and SIFT features
    feats = np.concatenate((hog_feats.ravel(), sift_feats.ravel()))

    # add the features to the list of all features
    all_feats.append(feats)

print(all(all_feats[0] == 0.0)) 

